In [1]:
#setup modules and l
import tensorflow as tf
import numpy as np
from functools import partial

from datetime import datetime

#now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
#root_logdir = "/Users/Tim/Development/ConvNN/logs/recoded_cnn"
#logdir = "{}/run-{}/".format(root_logdir,now)

In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
#derived in part from ageron and hvass
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_maps = 16
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_maps = 36
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"

n_fc1 = 128
fc1_dropout_rate = 0.5

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_maps, kernel_size=conv1_ksize,
                        strides=conv1_stride, padding=conv1_pad, activation=tf.nn.relu, name="conv1")
conv1_maxpool = tf.nn.max_pool(value=conv1, 
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
conv2 = tf.layers.conv2d(conv1_maxpool, filters=conv2_maps, kernel_size=conv2_ksize,
                        strides=conv2_stride, padding=conv2_pad, activation=tf.nn.relu, name="conv2")
conv2_maxpool = tf.nn.max_pool(value=conv2,
                               ksize=[1,2,2,1],
                               strides=[1,2,2,1],
                               padding="SAME", name="conv1_maxpool")
flat_layer = tf.reshape(conv2_maxpool, shape=[-1, conv2_maps * 7 * 7], name="flat_layer")

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(flat_layer, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_prob = tf.nn.softmax(logits, name="Y_prob")
    
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()




In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
n_epochs = 10
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    
    save_path = saver.save(sess, "/home/ec2-user/notebooks/Development/ConvNN/AWS_Models/recoded_cnn.ckpt")

0 Train accuracy: 0.98 Test accuracy: 0.983
1 Train accuracy: 0.98 Test accuracy: 0.9842
2 Train accuracy: 1.0 Test accuracy: 0.9837
3 Train accuracy: 0.98 Test accuracy: 0.9884
4 Train accuracy: 1.0 Test accuracy: 0.9904
5 Train accuracy: 1.0 Test accuracy: 0.9911
6 Train accuracy: 1.0 Test accuracy: 0.9913
7 Train accuracy: 1.0 Test accuracy: 0.9905
8 Train accuracy: 1.0 Test accuracy: 0.9882
9 Train accuracy: 1.0 Test accuracy: 0.9901
